Preparing to submit wold stranded samples....

This got a little messy because a couple libraries needed a top up.

In [1]:
import os
import sys
import requests
import pandas
import paramiko
import json
from IPython import display
from pathlib import Path

In [2]:
from curation_common import *
from htsworkflow.submission.encoded import DCCValidator

In [3]:
from htsworkflow.submission.encoded import Document
from htsworkflow.submission.aws_submission import run_aws_cp

In [4]:
# live server & control file
server = ENCODED('www.encodeproject.org')
spreadsheet_name = Path('~/woldlab/ENCODE/stranded-23171-23197-gastrocnemius.xlsx').expanduser()
engine=None
#engine='odf'

# test server & datafile
#server = ENCODED('test.encodedcc.org')
#spreadsheet_name = os.path.expanduser('~diane/woldlab/ENCODE/C1-encode3-limb-2017-testserver.ods')

server.load_netrc()
validator = DCCValidator(server)

In [5]:
award = 'UM1HG009443'

# Lookup biosample ontologies

Lookup any biosample ontologies that are already present

In [6]:
biosample = pandas.read_excel(spreadsheet_name, sheet_name='Biosample', header=0, engine=engine)

for i, row in biosample.iterrows():
    if not pandas.isnull(row.accession) and row.accession.startswith('E'):
        obj = server.get_json(row.accession)
        biosample_ontology = obj['biosample_ontology']
        if isinstance(biosample_ontology, dict):
            biosample.loc[i, 'biosample_ontology'] = biosample_ontology['@id']
            biosample.loc[i, 'biosample_term_name:skip'] = biosample_ontology['term_name']
            
biosample

,uuid,accession,aliases:array,biosample_ontology,biosample_term_name:skip,description,library_id:skip,cDNA_sample:skip,organism,source,lab,award,donor,Unnamed: 13
0,NaN,ENCBS895ZIH,barbara-wold:gastrocnemius_B6Cast_F1_PND4_Male...,/biosample-types/tissue_UBERON_0001388/,gastrocnemius,gastrocnemius_B6Cast F1 PND4_Male 03_ENC4_cDNA465,23171,ENC4_cDNA465,/organisms/mouse/,/sources/jackson-labs/,barbara-wold,UM1HG009443,/mouse-donors/ENCDO509HIY/,NaN
1,NaN,ENCBS659ATN,barbara-wold:gastrocnemius_B6Cast_F1_PND4_Male...,/biosample-types/tissue_UBERON_0001388/,gastrocnemius,gastrocnemius_B6Cast F1 PND4_Male 04_ENC4_cDNA466,23172,ENC4_cDNA466,/organisms/mouse/,/sources/jackson-labs/,barbara-wold,UM1HG009443,/mouse-donors/ENCDO509HIY/,NaN
2,NaN,ENCBS559PTX,barbara-wold:gastrocnemius_B6Cast_F1_PND4_Fema...,/biosample-types/tissue_UBERON_0001388/,gastrocnemius,gastrocnemius_B6Cast F1 PND4_Female 03_ENC4_cD...,23173,ENC4_cDNA467,/organisms/mouse/,/sources/jackson-labs/,barbara-wold,UM1HG009443,/mouse-donors/ENCDO509HIY/,NaN
3,NaN,ENCBS519KGT,barbara-wold:gastrocnemius_B6Cast_F1_PND4_Fema...,/biosample-types/tissue_UBERON_0001388/,gastrocnemius,gastrocnemius_B6Cast F1 PND4_Female 04_ENC4_cD...,23174,ENC4_cDNA468,/organisms/mouse/,/sources/jackson-labs/,barbara-wold,UM1HG009443,/mouse-donors/ENCDO509HIY/,NaN
4,NaN,ENCBS766LFK,barbara-wold:gastrocnemius_B6Cast_F1_PND10_Mal...,/biosample-types/tissue_UBERON_0001388/,gastrocnemius,gastrocnemius_B6Cast F1 PND10_Male 03_ENC4_cDN...,23175,ENC4_cDNA469,/organisms/mouse/,/sources/jackson-labs/,barbara-wold,UM1HG009443,/mouse-donors/ENCDO509HIY/,NaN
5,NaN,ENCBS078GAU,barbara-wold:gastrocnemius_B6Cast_F1_PND10_Mal...,/biosample-types/tissue_UBERON_0001388/,gastrocnemius,gastrocnemius_B6Cast F1 PND10_Male 04_ENC4_cDN...,23176,ENC4_cDNA470,/organisms/mouse/,/sources/jackson-labs/,barbara-wold,UM1HG009443,/mouse-donors/ENCDO509HIY/,NaN
6,NaN,ENCBS927BKO,barbara-wold:gastrocnemius_B6Cast_F1_PND10_Fem...,/biosample-types/tissue_UBERON_0001388/,gastrocnemius,gastrocnemius_B6Cast F1 PND10_Female 03_ENC4_c...,23177,ENC4_cDNA471,/organisms/mouse/,/sources/jackson-labs/,barbara-wold,UM1HG009443,/mouse-donors/ENCDO509HIY/,NaN
7,NaN,ENCBS900JGS,barbara-wold:gastrocnemius_B6Cast_F1_PND10_Fem...,/biosample-types/tissue_UBERON_0001388/,gastrocnemius,gastrocnemius_B6Cast F1 PND10_Female 04_ENC4_c...,23178,ENC4_cDNA472,/organisms/mouse/,/sources/jackson-labs/,barbara-wold,UM1HG009443,/mouse-donors/ENCDO509HIY/,NaN
8,NaN,ENCBS340HGO,barbara-wold:gastrocnemius_B6Cast_F1_PND14_3x_...,/biosample-types/tissue_UBERON_0001388/,gastrocnemius,gastrocnemius_B6Cast F1 PND14_3x Males 03_ENC4...,23179,ENC4_cDNA473,/organisms/mouse/,/sources/jackson-labs/,barbara-wold,UM1HG009443,/mouse-donors/ENCDO509HIY/,NaN
9,NaN,ENCBS368GVV,barbara-wold:gastrocnemius_B6Cast_F1_PND14_3x_...,/biosample-types/tissue_UBERON_0001388/,gastrocnemius,gastrocnemius_B6Cast F1 PND14_3x Males 04_ENC4...,23180,ENC4_cDNA474,/organisms/mouse/,/sources/jackson-labs/,barbara-wold,UM1HG009443,/mouse-donors/ENCDO509HIY/,NaN


# Register Biosamples

In [7]:
biosamples = pandas.read_excel(spreadsheet_name, sheet_name='Biosample', header=0, engine=engine)
created = server.post_sheet('/biosamples/', biosamples, 
                            verbose=True,
                            dry_run=True, 
                            validator=validator)
print(len(created))

0


In [8]:
if created:
    biosample.to_excel('/dev/shm/biosamples.xlsx', index=False)

In [1]:
## Forgot to post model_organism_sex

In [ ]:
print(spreadsheet_name)
biosamples = pandas.read_excel(spreadsheet_name, sheet_name='Biosample', header=0, engine=engine)
for i, row in biosamples.iterrows():
    biosample = server.get_json(row['accession'])
    model_organism_age = biosample.get('model_organism_sex')
    if model_organism_age != row['model_organism_sex']:
        print("{} is {} should be {}".format(row['accession'], model_organism_age, row['model_organism_sex']))
        #print(server.patch_json(
        #    '/biosamples/{}/'.format(row['accession']),
        #    {'model_organism_sex': row['model_organism_sex']}))

# Register Libraries

In [9]:
print(spreadsheet_name)
libraries = pandas.read_excel(spreadsheet_name, sheet_name='Library', header=0, engine=engine)
created = server.post_sheet('/libraries/', 
                            libraries, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

/home/diane/woldlab/ENCODE/stranded-23171-23197-gastrocnemius.xlsx
0


In [10]:
if created:
    libraries.to_excel('/dev/shm/libraries.xlsx', index=False)

# Register Experiments

In [11]:
print(server.server)
experiments = pandas.read_excel(spreadsheet_name, sheet_name='Experiment', header=0, engine=engine)
experiments = experiments[experiments['accession'] != 'barbara approval needed']
created = server.post_sheet('/experiments/', 
                            experiments, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

www.encodeproject.org
0


In [12]:
if created:
    experiments.to_excel('/dev/shm/experiments.xlsx', index=False)

In [13]:
print(server.server)
experiments = pandas.read_excel(spreadsheet_name, sheet_name='Experiment', header=0, engine=engine)

#for i, row in experiments.iterrows():
#    obj = server.get_json(row['accession'])
#    updates = {}
#    if obj['aliases'] != row['aliases:array']:
#        updates['aliases'] = [row['aliases:array']]
#    if obj['description'] != row['description']:
#        updates['description'] = row['description']

#    if len(updates) > 0:
#        print(obj['@id'], updates)
        #print(server.patch_json(obj['@id'], updates))

www.encodeproject.org


# Register Replicates

In [22]:
print(server.server)
print(spreadsheet_name)
replicates = pandas.read_excel(spreadsheet_name, sheet_name='Replicate', header=0, engine=engine)
replicates = replicates[replicates['uuid'] != 'barbara approval needed']
created = server.post_sheet('/replicates/',
                            replicates, 
                            verbose=True,
                            dry_run=False, 
                            validator=validator)
print(len(created))

www.encodeproject.org
/home/diane/woldlab/ENCODE/stranded-23171-23197-gastrocnemius.xlsx
Reponse {'status': 'success', '@type': ['result'], '@graph': [{'date_created': '2021-04-16T16:51:22.293230+00:00', 'submitted_by': '/users/bc5b62f7-ce28-4a1e-b6b3-81c9c5a86d7a/', 'aliases': ['barbara-wold:23171_b1_t1'], 'schema_version': '9', 'biological_replicate_number': 1, 'technical_replicate_number': 1, 'experiment': '/experiments/ENCSR102ZZC/', 'library': '/libraries/ENCLB961GWJ/', 'status': 'in progress', '@id': '/replicates/ff42175b-e174-4d15-bc5b-3b31ccaba8f3/', '@type': ['Replicate', 'Item'], 'uuid': 'ff42175b-e174-4d15-bc5b-3b31ccaba8f3'}]}
row 0 created: ff42175b-e174-4d15-bc5b-3b31ccaba8f3
Reponse {'status': 'success', '@type': ['result'], '@graph': [{'date_created': '2021-04-16T16:51:22.592089+00:00', 'submitted_by': '/users/bc5b62f7-ce28-4a1e-b6b3-81c9c5a86d7a/', 'aliases': ['barbara-wold:23172_b2_t1'], 'schema_version': '9', 'biological_replicate_number': 2, 'technical_replicate_num

Reponse {'status': 'success', '@type': ['result'], '@graph': [{'date_created': '2021-04-16T16:51:25.959739+00:00', 'submitted_by': '/users/bc5b62f7-ce28-4a1e-b6b3-81c9c5a86d7a/', 'aliases': ['barbara-wold:23185_b1_t1'], 'schema_version': '9', 'biological_replicate_number': 1, 'technical_replicate_number': 1, 'experiment': '/experiments/ENCSR510HZQ/', 'library': '/libraries/ENCLB666MVG/', 'status': 'in progress', '@id': '/replicates/f309bf11-bb7a-491c-9c0c-fa5439d472ad/', '@type': ['Replicate', 'Item'], 'uuid': 'f309bf11-bb7a-491c-9c0c-fa5439d472ad'}]}
row 14 created: f309bf11-bb7a-491c-9c0c-fa5439d472ad
Reponse {'status': 'success', '@type': ['result'], '@graph': [{'date_created': '2021-04-16T16:51:26.237385+00:00', 'submitted_by': '/users/bc5b62f7-ce28-4a1e-b6b3-81c9c5a86d7a/', 'aliases': ['barbara-wold:23186_b1_t1'], 'schema_version': '9', 'biological_replicate_number': 1, 'technical_replicate_number': 1, 'experiment': '/experiments/ENCSR640PLU/', 'library': '/libraries/ENCLB428JLV/'

In [23]:
if created:
    replicates.to_excel('/dev/shm/replicates.xlsx', index=False)

# Check Files

In [20]:
files = pandas.read_excel(spreadsheet_name, sheet_name='File', header=0, engine=engine)
created = server.post_sheet('/files/', files, verbose=True, dry_run=True, validator=validator)
print(len(created))

27
